In [1]:
import xarray as xr
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline
xr.__version__

'0.11.1+65.g627a881'

In [2]:
import zarr
zarr.__version__

'2.2.1.dev146'

- don't forget to do some port forwarding from the machine your browser is running on:
- ssh -L 8787:localhost:8787 naomi@mary
- (and DASK DASHBOARD URL = http://127.0.0.1:8787/status)
            

In [3]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:45658 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 40 Memory: 134.90 GB


In [9]:
from esgf2zarr.aggregate import set_bnds_as_coords


In [10]:
#%%time
ncfile1 = '/dm5/naomi/CMIP6mon/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/uo/gn/v20181129/uo_Omon_BCC-ESM1_historical_r1i1p1f1_gn_185001-185912.nc'
ds1 = xr.open_mfdataset(ncfile1, decode_times=False, preprocess=set_bnds_as_coords)
ncfile2 = '/dm5/naomi/CMIP6mon/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/uo/gn/v20181129/uo_Omon_BCC-ESM1_historical_r1i1p1f1_gn_186001-186912.nc'
ds2 = xr.open_mfdataset(ncfile2, decode_times=False, preprocess=set_bnds_as_coords)

In [11]:
print(ds1.time.attrs['units'],' and ',ds2.time.attrs['units'])

days since 1850-1-1  and  days since 1860-1-1


In [12]:
ds = xr.open_mfdataset([ncfile1,ncfile2], decode_times=True, preprocess=set_bnds_as_coords)
ds.coords

Coordinates:
  * lev        (lev) float64 5.0 15.0 25.0 ... 4.289e+03 4.807e+03 5.334e+03
    lev_bnds   (lev, bnds) float64 0.0 10.0 10.0 ... 5.071e+03 5.071e+03 5.6e+03
  * lat        (lat) float64 -81.0 -80.0 -79.0 -78.0 ... 87.0 88.0 89.0 90.0
    lat_bnds   (lat, bnds) float64 -81.0 -80.5 -80.5 -79.5 ... 89.5 89.5 90.0
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    lon_bnds   (lon, bnds) float64 -0.5 0.5 0.5 1.5 ... 357.5 358.5 358.5 359.5
    latitude   (lat, lon) float32 -81.0 -81.0 -81.0 ... 88.4463 88.2237 88.00084
    longitude  (lat, lon) float32 0.0 1.0 2.0 3.0 4.0 ... 80.0 80.0 80.0 80.0
  * time       (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<shape=(240, 2), chunksize=(120, 2)>

In [13]:
ds['uo'].chunk(chunks={'time' : 'auto'})

<xarray.DataArray 'uo' (time: 240, lev: 40, lat: 232, lon: 360)>
dask.array<shape=(240, 40, 232, 360), dtype=float32, chunksize=(10, 40, 232, 360)>
Coordinates:
  * lev        (lev) float64 5.0 15.0 25.0 ... 4.289e+03 4.807e+03 5.334e+03
  * lat        (lat) float64 -81.0 -80.0 -79.0 -78.0 ... 87.0 88.0 89.0 90.0
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    latitude   (lat, lon) float32 dask.array<shape=(232, 360), chunksize=(232, 360)>
    longitude  (lat, lon) float32 dask.array<shape=(232, 360), chunksize=(232, 360)>
  * time       (time) object 1850-01-16 12:00:00 ... 1869-12-16 12:00:00
Attributes:
    standard_name:  sea_water_x_velocity
    long_name:      Sea Water X Velocity
    comment:        Prognostic x-ward velocity component resolved by the model.
    units:          m s-1
    original_name:  uo
    cell_methods:   time: mean (interval: 30 minutes)